# Sample budget calculated with the simulation

In [110]:
# simple trick to make sure that we work in the same directory when we are inside a jupyter notebook and when we are running the simulation from the main directory
import os
import pandas as pd
import datetime
cwd = os.getcwd()
if cwd.endswith('jupyter_notebooks'):
    os.chdir('..')

In [111]:
all_data = pd.read_csv('out.csv', dtype={'CF': str}, converters={'date': pd.to_datetime})

In [112]:
monthly_data = all_data.drop(columns=['CF', 'rule', 'note']).groupby('date').sum()
monthly_data.reset_index(inplace=True)

## display the monthly budget

In [113]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()
from bokeh.models import ColumnDataSource, CustomJS, NumeralTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.models.widgets import RangeSlider
from bokeh.layouts import column

# Base viz configuration
source = ColumnDataSource(monthly_data)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
p=figure(y_range=(0, monthly_data['budget'].max()), x_range=(monthly_data['date'].min(),monthly_data['date'].max()), x_axis_type='datetime', tools=TOOLS)
p.title.text = "budget simulation (from {} to {})".format(monthly_data['date'].min().strftime("%d %b %Y"), monthly_data['date'].max().strftime("%d %b %Y"))
p.xaxis.axis_label = "date"
p.yaxis.axis_label = "budget"
p.yaxis.formatter=NumeralTickFormatter(format="0,0")

# Data line
p.line(x='date', y='budget', source=source, line_width=2)

# Hover
hover=HoverTool()
hover.tooltips = [
    ('Date', "@date{%F}"),
    ('Monthly budget', "@budget{'0,0'} CHF"),
]
hover.formatters={
    'date': 'datetime',
    'budget': 'numeral',
}
hover.mode='vline'
p.add_tools(hover)

# Rendering of the viz
layout=column(p, sizing_mode='stretch_width')
show(layout)

Loading BokehJS ...

## Display the yearly budget

In [114]:
yearly_data = monthly_data.copy(deep=True)
yearly_data['year'] = yearly_data['date'].dt.year

yearly_data.drop(columns=['date'], inplace=True)
yearly_data = yearly_data.groupby('year').sum()
yearly_data.reset_index(inplace=True)


In [115]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()
from bokeh.models import ColumnDataSource, CustomJS, NumeralTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.models.widgets import RangeSlider
from bokeh.layouts import column

# Base viz configuration
source2 = ColumnDataSource(yearly_data)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
p2=figure(y_range=(0, yearly_data['budget'].max()), x_range=(yearly_data['year'].min(),yearly_data['year'].max()), tools=TOOLS)
p2.title.text = "Yearly budget simulation (from {} to {})".format(yearly_data['year'].min(), yearly_data['year'].max())
p2.xaxis.axis_label = "year"
p2.yaxis.axis_label = "budget"
p2.yaxis.formatter=NumeralTickFormatter(format="0,0")

# Data line
p2.line(x='year', y='budget', source=source2, line_width=2)

# Hover
hover=HoverTool()
hover.tooltips = [
    ('Year', "@year"),
    ('Yearly budget', "@budget{'0,0'} CHF"),
]
hover.formatters={
    'year': 'numeral',
    'budget': 'numeral',
}
hover.mode='vline'
p2.add_tools(hover)

# Rendering of the viz
layout=column(p2, sizing_mode='stretch_width')
show(layout)

Loading BokehJS ...